In [131]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode, binom, poisson, norm, expon, f_oneway, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import AnovaRM
import stat_240829 as stat

In [41]:
from statsmodels.stats.multitest import multipletests

In [85]:
import itertools

# Descriptive Statistics: describe()

In [26]:
np.random.seed(42)
df = pd.DataFrame({
    'Var1': np.random.rand(20),
    'Var2': np.random.rand(20),
    'Var3': np.random.rand(20),
    'Var4': np.random.rand(20),
    'Var5': np.random.rand(20),
    'c': ['X', 'X', 'Y', 'Z', 'Y', 'X', 'Z', 'Y', 'Z', 'Y','A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'Y'],
    'a': ['pre', 'min', 'Y', 'pre', 'Y', 'post', 'min', 'Y', 'post', 'Y','pre', 'post', 'min', 'B', 'B', 'B', 'C', 'C', 'C', 'Y']
})

In [17]:
stat.describe(df,cols=['Var1','Var2','Var3','Var4','Var5'])

,mean,median,variance,std_dev,mad,min,max,range,skewness,kurtosis,count,sum,25%,50%,75%
Var1,0.457702,0.403243,0.094671,0.307687,0.234321,0.020584,0.969910,0.949325,0.277863,-1.263481,20,9.154046,0.183010,0.403243,0.714053
Var2,0.454829,0.448111,0.084785,0.291179,0.242280,0.046450,0.965632,0.919182,0.227964,-1.084160,20,9.096579,0.192386,0.448111,0.629948
Var3,0.489971,0.507622,0.109425,0.330794,0.317204,0.034389,0.969585,0.935196,0.116779,-1.421157,20,9.799421,0.193201,0.507622,0.805056
Var4,0.461058,0.373572,0.098833,0.314378,0.299274,0.005522,0.986887,0.981365,0.098072,-1.460762,20,9.221165,0.184268,0.373572,0.771514
Var5,0.487343,0.508264,0.072195,0.268691,0.201131,0.025419,0.887213,0.861794,-0.279719,-1.048767,20,9.746863,0.321633,0.508264,0.717335


# Statistical Test Flow Chart: test()

In [ ]:
### Revising because this is too much ###
''' test: Statistical test flow chart
        question: difference, correlation, categories
        factors: # of factors
        same: same (True) or different (False) subjects
        compare: list of comparisons
        para: parameteric (True) or nonparametric (False)
    Dependencies:
'''
def test(df: pd.DataFrame(), compare: list(), question: str(), factors=1, same=False, para=True):
    return ''

need to reformat to tidy data and then it will be good for pairwise turkey_hsd

In [141]:
''' difference: Statistical test flow chart
        df: tidy DataFrame
        data_col: data column name
        compare_col: comparisons column name
        compare: list of comparisons
        same: same (True) or different (False) subjects
        compare: list of comparisons
        para: parameteric (True) or nonparametric (False)
        alpha: significance level for the test (Default: 0.05)
        within_cols: list of column names corresponding to different conditions or time points with the same subject (optional; para=True, same=True)
        method: multiple hypothesis testing correction method (Default: holm)
    Dependencies: pandas,scipy.stats,statsmodels.stats
'''
def difference(df: pd.DataFrame(), data_col: str(), compare_col: str(), compare: list(), same=False, para=True, alpha=0.05, within_cols=[],method='holm'):
    
    if not same: # Different samples

        if para==True: # Data that follows a normal distribution

            if len(compare)==2: # Comparing 2 samples

                # Student's T Test Calculation
                print('Statistical Test: Student\'s T-test \n - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).')
                t_stat, p_value = ttest_ind(*(df[df[compare_col]==comp][data_col] for comp in compare))
                if p_value<alpha: null='reject'
                else: null='fail to reject'
                inference = pd.DataFrame({'test':['Student\'s T-test'],
                                            'comparison': [','.join(compare)], 
                                            'p_value': [p_value],
                                            'null_hypothesis':[null]})
                
                return inference
            
            elif len(compare)>2: # Comparing more than 2 samples
                
                # 1-way Anova Test
                print('Statistical Test: 1-way Anova\n - If you want to compare all three groups simultaneously to determine if there is a significant difference in their means.\n - If the ANOVA finds a significant difference, then perform post-hoc tests to determine which specific groups differ from each other.')
                f_stat, p_value = f_oneway(*(df[df[compare_col]==comp][data_col] for comp in compare))
                if p_value<alpha: null='reject'
                else: null='fail to reject'
                inference = pd.DataFrame({'test':['1-way Anova Test'],
                                            'comparison': [','.join(compare)],
                                            'p_value':[p_value],
                                            'null_hypothesis': [null]})
                
                # Tukey's Honestly Significant Difference Test
                print('Statistical Test: Tukey\'s Honestly Significant Difference (HSD) Test\n - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.\n - Use with equal group sizes.')
                df2 = pd.concat([df[df[compare_col]==comp] for comp in compare]).reset_index(drop=True) # Only include specified data
                tukey_result = pairwise_tukeyhsd(df2[data_col], df2[compare_col], alpha=alpha)
                tukey_df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])
                inference = pd.concat([inference,
                                        pd.DataFrame({'test':['Tukey\'s HSD Test']*len(compare),
                                                        'comparison': [','.join([tukey_df.iloc[i]['group1'],tukey_df.iloc[i]['group2']]) for i in range(len(tukey_df))], 
                                                        'p_value': tukey_df['p-adj'],
                                                        'null_hypothesis': ['reject' if p_adj<alpha else 'fail to reject' for p_adj in tukey_df['p-adj']]})]).reset_index(drop=True)

                # Multiple Hypothesis Correction for Student's T Test
                print(f'Statistical Test: Student\'s Paired T-test ({method} correction) \n - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.\n - Use with unequal group sizes.')
                tests = list(itertools.combinations(df2[compare_col].unique(), 2)) # Generate all comparisons from unique conditions
                p_values = [] # Perform pairwise t-tests
                comparisons = []
                for (cond1, cond2) in tests:
                    t_stat, p_value = ttest_ind(df2[df2[compare_col] == cond1][data_col], df2[df2[compare_col] == cond2][data_col])
                    p_values.append(p_value)
                    comparisons.append(','.join([cond1,cond2]))
                corrected_p_values = list(multipletests(p_values, method=method)[0]) # Apply Bonferroni correction
                inference = pd.concat([inference,
                                        pd.DataFrame({'test':[f'Student\'s T-test ({method} correction)']*len(tests),
                                                        'comparison': comparisons, 
                                                        'p_value': p_values,
                                                        'null_hypothesis':['reject' if corrected_p_value else 'fail to reject' for corrected_p_value in corrected_p_values]})]).reset_index(drop=True)

                return inference
            
            else: print('Error: Invalid compare. List needs to contain 2 or more stings')
        
        else: # Data that does not follow a normal distribution
            
            # Mann Whitney U Test
            print(f'Statistical Test: Mann Whitney U Test ({method} correction)\n - The Mann-Whitney U test is a non-parametric test used to compare differences between two independent groups when the dependent variable is either ordinal or continuous, but not normally distributed.')
            df2 = pd.concat([df[df[compare_col]==comp] for comp in compare]).reset_index(drop=True) # Only include specified data
            tests = list(itertools.combinations(df2[compare_col].unique(), 2)) # Generate all test comparisons from unique conditions
            p_values = []
            comparisons = []
            for (cond1, cond2) in tests:
                u_stat, p_value = mannwhitneyu(df2[df2[compare_col] == cond1][data_col], df2[df2[compare_col] == cond2][data_col], alternative='two-sided')
                p_values.append(p_value)
                comparisons.append(','.join([cond1,cond2]))
            corrected_p_values = list(multipletests(p_values, method=method)[0]) # Apply Bonferroni correction
            inference = pd.DataFrame({'test':['Mann Whitney U Test' if len(tests)==1 else f'Mann Whitney U Test ({method} correction)']*len(tests),
                                        'comparison': comparisons, 
                                        'p_value': p_values,
                                        'null_hypothesis':['reject' if corrected_p_value else 'fail to reject' for corrected_p_value in corrected_p_values]})

            return inference

    else: # Same sample

        if para==True:  # Data that follows a normal distribution

            if len(compare)==2: # Comparing 2 related (paired) samples
                
                # Student's Paired T-test
                print('Statistical Test: Paired Student\'s T-test \n - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).\n - Assumes the two groups are related or paired, meaning that each data point in one group has a corresponding data point in the other group.')
                t_stat, p_value = ttest_rel(*(df[df[compare_col]==comp][data_col] for comp in compare))
                if p_value<alpha: null='reject'
                else: null='fail to reject'
                inference = pd.DataFrame({'test':['Paired Student\'s T-test'],
                                            'comparison': [','.join(compare)], 
                                            'p_value': [p_value],
                                            'null_hypothesis':[null]})
                
                return inference

            elif len(compare)>2: # Comparing 2 or more related (paired) samples
                
                # Repeated Anova
                print('Statistical Test: Repeated 1-way Anova\n - Use repeated measures ANOVA when you have multiple measurements from the same subjects or units under different conditions or time points.\n - It is particularly useful when the goal is to reduce error variability and account for within-subject correlations, thereby increasing the power of the test.')
                df2 = pd.concat([df[df[compare_col]==comp] for comp in compare]).reset_index(drop=True) # Only include specified data
                anova = AnovaRM(data=df2, depvar=data_col, subject=compare_col,within=within_cols)
                anova_result = anova.fit()
                anova_df = anova_result.summary().tables[0]
                inference = pd.DataFrame({'test':['Repeated Anova']*len(within_cols),
                                            'comparison': [','.join(compare) + f'; Within = {w}' for w in within_cols], 
                                            'p_value': [p_val for p_val in anova_df['Pr > F']],
                                            'null_hypothesis':['reject' if p_val<alpha else 'fail to reject' for p_val in anova_df['Pr > F']]})


                # Multiple Hypotheis Correction for Student's Paired T-test
                print(f'Statistical Test: Student\'s Paired T-test ({method} correction) \n - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.')
                for w in within_cols: # Iterate through within subject columns
                    tests = list(itertools.combinations(df2[w].unique(), 2)) # Generate all pairwise comparisons from unique conditions
                    p_values = [] # Perform pairwise t-tests
                    comparisons = []
                    for (cond1, cond2) in tests:
                        t_stat, p_value = ttest_rel(df2[df2[w] == cond1][data_col], df2[df2[w] == cond2][data_col])
                        p_values.append(p_value)
                        comparisons.append(','.join(compare) + f'; Within = {w}; ' + ','.join([cond1,cond2]))
                    corrected_p_values = list(multipletests(p_values, method=method)[0]) # Apply Bonferroni correction
                    inference = pd.concat([inference,
                                            pd.DataFrame({'test':[f'Student\'s Paired T-test ({method} correction)']*len(tests),
                                                            'comparison': comparisons, 
                                                            'p_value': p_values,
                                                            'null_hypothesis':['reject' if corrected_p_value else 'fail to reject' for corrected_p_value in corrected_p_values]})]).reset_index(drop=True)
                
                return inference

            else: print('Error: Invalid compare. List needs to contain 2 or more stings')

        else: # Data that does not follow a normal distribution
            
            # Wilcoxon Signed-Rank Test
            print(f'Statistical Test: Wilcoxon Signed-Rank Test ({method} correction) \n - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.')
            df2 = pd.concat([df[df[compare_col]==comp] for comp in compare]).reset_index(drop=True) # Only include specified data
            tests = list(itertools.combinations(df2[compare_col].unique(), 2)) # Generate all pairwise comparisons from unique conditions
            p_values = [] # Perform pairwise t-tests
            comparisons = []
            for (cond1, cond2) in tests:
                w_stat, p_value = wilcoxon(df2[df2[compare_col] == cond1][data_col], df2[df2[compare_col] == cond2][data_col])
                p_values.append(p_value)
                comparisons.append(','.join([cond1,cond2]))
            corrected_p_values = list(multipletests(p_values, method=method)[0]) # Apply Bonferroni correction
            inference = pd.DataFrame({'test':['Wilcoxon Signed-Rank Test' if len(tests)==1 else f'Wilcoxon Signed-Rank Test ({method} correction)']*len(tests),
                                        'comparison': comparisons, 
                                        'p_value': p_values,
                                        'null_hypothesis':['reject' if corrected_p_value else 'fail to reject' for corrected_p_value in corrected_p_values]})
            
            return inference

In [139]:
difference(df=df,data_col='Var5',compare_col='c',compare=['A','B','C'],factors=1,same=True,para=False)

Statistical Test: Wilcoxon Signed-Rank Test (holm correction) 
 - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.


,test,comparison,p_value,null_hypothesis
0,Wilcoxon Signed-Rank Test (holm correction),"A,B",0.75,fail to reject
1,Wilcoxon Signed-Rank Test (holm correction),"A,C",0.75,fail to reject
2,Wilcoxon Signed-Rank Test (holm correction),"B,C",0.25,fail to reject


In [140]:
difference(df=df,data_col='Var5',compare_col='c',compare=['A','B'],factors=1,same=True,para=False)

Statistical Test: Wilcoxon Signed-Rank Test (holm correction) 
 - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.


,test,comparison,p_value,null_hypothesis
0,Wilcoxon Signed-Rank Test,"A,B",0.75,fail to reject


In [129]:
difference(df=df,data_col='Var5',compare_col='c',compare=['X','Y','Z'],factors=1,same=False,para=False)

Statistical Test: Mann Whitney U Test (holm correction)
 - The Mann-Whitney U test is a non-parametric test used to compare differences between two independent groups when the dependent variable is either ordinal or continuous, but not normally distributed.


,test,comparison,p_value,null_hypothesis
0,Mann Whitney U Test (holm correction),"X,Y",0.392857,fail to reject
1,Mann Whitney U Test (holm correction),"X,Z",1.000000,fail to reject
2,Mann Whitney U Test (holm correction),"Y,Z",0.571429,fail to reject


In [122]:
difference(df=df,data_col='Var5',compare_col='c',compare=['X','Z'],factors=1,same=False,para=False)

Statistical Test: Mann Whitney U Test (holm correction)
 - The Mann-Whitney U test is a non-parametric test used to compare differences between two independent groups when the dependent variable is either ordinal or continuous, but not normally distributed.


,test,comparison,p_value,null_hypothesis
0,Mann Whitney U Test,"X,Z",1.0,fail to reject


In [123]:
difference(df=df,data_col='Var5',compare_col='c',compare=['X','Z','A'],factors=1,same=True,para=True,within_cols=['a'])

Statistical Test: Repeated 1-way Anova
 - Use repeated measures ANOVA when you have multiple measurements from the same subjects or units under different conditions or time points.
 - It is particularly useful when the goal is to reduce error variability and account for within-subject correlations, thereby increasing the power of the test.
Statistical Test: Student's Paired T-test (holm correction) 
 - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.


,test,comparison,p_value,null_hypothesis
0,Repeated Anova,"X,Z,A; Within = a",0.523434,fail to reject
1,Student's Paired T-test (holm correction),"X,Z,A; Within = a; pre,min",0.354758,fail to reject
2,Student's Paired T-test (holm correction),"X,Z,A; Within = a; pre,post",0.558124,fail to reject
3,Student's Paired T-test (holm correction),"X,Z,A; Within = a; min,post",0.681190,fail to reject


In [124]:
difference(df=df,data_col='Var5',compare_col='c',compare=['X','Y'],factors=1,same=False,para=True)

Statistical Test: Student's T-test 
 - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).


,test,comparison,p_value,null_hypothesis
0,Student's T-test,"X,Y",0.205944,fail to reject


In [130]:
difference(df=df,data_col='Var5',compare_col='c',compare=['X','Y','Z'],factors=1,same=False,para=True)

Statistical Test: 1-way Anova
 - If you want to compare all three groups simultaneously to determine if there is a significant difference in their means.
 - If the ANOVA finds a significant difference, then perform post-hoc tests to determine which specific groups differ from each other.
Statistical Test: Tukey's Honestly Significant Difference (HSD) Test
 - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.
 - Use with equal group sizes.
Statistical Test: Student's Paired T-test (holm correction) 
 - To control for Type-I error, adjust the significance threshold (α) to account for the number of tests.
 - Use with unequal group sizes.


,test,comparison,p_value,null_hypothesis
0,1-way Anova Test,"X,Y,Z",0.515145,fail to reject
1,Tukey's HSD Test,"X,Y",0.547800,fail to reject
2,Tukey's HSD Test,"X,Z",0.981700,fail to reject
3,Tukey's HSD Test,"Y,Z",0.665900,fail to reject
4,Student's T-test (holm correction),"X,Y",0.205944,fail to reject
5,Student's T-test (holm correction),"X,Z",0.889875,fail to reject
6,Student's T-test (holm correction),"Y,Z",0.422474,fail to reject


In [14]:
difference(df=df,data_col='Var5',compare_col='c',compare=['A','B'],factors=1,same=True,para=True)

Statistical Test: Paired Student's T-test 
  - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).
 - Assumes the two groups are related or paired, meaning that each data point in one group has a corresponding data point in the other group.


,test,comparison,p_value,null_hypothesis
0,Paired Student's T-test,"A,B",0.741616,fail to reject


### Correlations

In [142]:
import pandas as pd

# Example DataFrame with n-dimensional data
data = pd.DataFrame({
    'X1': [1, 2, 3, 4, 5],
    'X2': [2, 3, 4, 5, 6],
    'X3': [5, 6, 7, 8, 9]
})

# Calculate correlation matrix
correlation_matrix = data.corr()

print(correlation_matrix)

     X1   X2   X3
X1  1.0  1.0  1.0
X2  1.0  1.0  1.0
X3  1.0  1.0  1.0


In [143]:
data.corr?

Signature:
data.corr(
    method: 'CorrelationMethod' = 'pearson',
    min_periods: 'int' = 1,
    numeric_only: 'bool' = False,
) -> 'DataFrame'
Docstring:
Compute pairwise correlation of columns, excluding NA/null values.

Parameters
----------
method : {'pearson', 'kendall', 'spearman'} or callable
    Method of correlation:

    * pearson : standard correlation coefficient
    * kendall : Kendall Tau correlation coefficient
    * spearman : Spearman rank correlation
    * callable: callable with input two 1d ndarrays
        and returning a float. Note that the returned matrix from corr
        will have 1 along the diagonals and will be symmetric
        regardless of the callable's behavior.
min_periods : int, optional
    Minimum number of observations required per pair of columns
    to have a valid result. Currently only available for Pearson
    and Spearman correlation.
numeric_only : bool, default False
    Include only `float`, `int` or `boolean` data.

    .. versionadded

In [167]:
''' correlation: Returns a correlation matrix
        df: DataFrame
        num_cols: list of numerical columns to compute statistics (optional)
        group: column name to split tidy dataframes (optional)
        method: pearson, spearman, or kendall (Default: pearson)
        numeric_only: only calculates correlations for numeric columns (Default: True)
    Depedencies: pandas
'''
def correlation(df: pd.DataFrame, cols=[], group='', method='pearson',numeric_only=True):


    if group!='': df = df.pivot(index=group,columns=cols) # Splits tidy dataframe
    if len(cols)>0: df = df[cols] # Isolate specified columns
    print(df)
    return df.corr(method=method,numeric_only=numeric_only) # Correlation matrix with specified method

In [150]:
correlation(df=df,cols=['Var1','Var2'])

,Var1,Var2
Var1,1.000000,-0.589734
Var2,-0.589734,1.000000


In [152]:
correlation(df=df,cols=['Var1','Var2','Var5'])

,Var1,Var2,Var5
Var1,1.000000,-0.589734,0.254500
Var2,-0.589734,1.000000,-0.063268
Var5,0.254500,-0.063268,1.000000


In [153]:
df

,Var1,Var2,Var3,Var4,Var5,c,a
0,0.374540,0.611853,0.122038,0.388677,0.863103,X,pre
1,0.950714,0.139494,0.495177,0.271349,0.623298,X,min
2,0.731994,0.292145,0.034389,0.828738,0.330898,Y,Y
3,0.598658,0.366362,0.909320,0.356753,0.063558,Z,pre
4,0.156019,0.456070,0.258780,0.280935,0.310982,Y,Y
5,0.155995,0.785176,0.662522,0.542696,0.325183,X,post
6,0.058084,0.199674,0.311711,0.140924,0.729606,Z,min
7,0.866176,0.514234,0.520068,0.802197,0.637557,Y,Y
8,0.601115,0.592415,0.546710,0.074551,0.887213,Z,post
9,0.708073,0.046450,0.184854,0.986887,0.472215,Y,Y


In [154]:
correlation(df=df)

,Var1,Var2,Var3,Var4,Var5
Var1,1.000000,-0.589734,-0.031893,-0.169203,0.254500
Var2,-0.589734,1.000000,0.275677,0.360417,-0.063268
Var3,-0.031893,0.275677,1.000000,0.001106,0.061004
Var4,-0.169203,0.360417,0.001106,1.000000,-0.140020
Var5,0.254500,-0.063268,0.061004,-0.140020,1.000000


In [155]:
correlation(df=df,cols=['Var1','Var2','Var5'],method='spearman')

,Var1,Var2,Var5
Var1,1.000000,-0.566917,0.246617
Var2,-0.566917,1.000000,-0.030075
Var5,0.246617,-0.030075,1.000000


In [156]:
correlation(df=df,cols=['Var1','Var2','Var5'],method='kendall')

,Var1,Var2,Var5
Var1,1.000000,-0.389474,0.168421
Var2,-0.389474,1.000000,-0.021053
Var5,0.168421,-0.021053,1.000000


In [164]:
correlation(df=df,cols=['Var1'],method='spearman',group='c')

Var1                         
c         A    B    C    X    Y    Z
     c                              
Var1 A  1.0  NaN  NaN  NaN  NaN  NaN
     B  NaN  1.0  NaN  NaN  NaN  NaN
     C  NaN  NaN  1.0  NaN  NaN  NaN
     X  NaN  NaN  NaN  1.0  NaN  NaN
     Y  NaN  NaN  NaN  NaN  1.0  NaN
     Z  NaN  NaN  NaN  NaN  NaN  1.0

In [162]:
import pyMUZ.gen.io as io
lc = io.get('/Users/marczepeda/Documents/Liau_Lab/Projects/2.ZF_degraders/1.Molecular_Biology/MUZ120/NGS/out/LC_outcomes.csv')
lc

,sample,edit,count,fraction
0,MUZ120-38_S38_L001_R1_001,WT,2712,0.546995
1,MUZ120-38_S38_L001_R1_001,H167R,2145,0.432634
2,MUZ120-38_S38_L001_R1_001,"H163P, H167R",4,0.000807
3,MUZ120-38_S38_L001_R1_001,"H167R, G169R",4,0.000807
4,MUZ120-38_S38_L001_R1_001,"R143Q, H167R",3,0.000605
...,...,...,...,...
1563,MUZ120-34_S34_L001_R1_001,G169W,1,0.000232
1564,MUZ120-34_S34_L001_R1_001,"N159T, H191Y",1,0.000232
1565,MUZ120-34_S34_L001_R1_001,N179I,1,0.000232
1566,MUZ120-34_S34_L001_R1_001,"F145L, N159T",1,0.000232


In [186]:
''' correlation: Returns a correlation matrix
        df: DataFrame
        num_cols: list of numerical columns to compute statistics (optional)
        group: column name to split tidy dataframes (optional)
        method: pearson, spearman, or kendall (Default: pearson)
        numeric_only: only calculates correlations for numeric columns (Default: True)
    Depedencies: pandas
'''
def correlation(df: pd.DataFrame, var_cols=[], value_cols=[], method='pearson',numeric_only=True):

    if (len(var_cols)==2)&(len(value_cols)==1): df = df.pivot(index=var_cols[0],columns=var_cols[1],values=value_cols[0]) # Splits tidy dataframe
    elif len(value_cols)>=1: df = df[value_cols] # Isolate specified columns for non-tidy dataframe
    return df.corr(method=method,numeric_only=numeric_only) # Correlation matrix with specified method

In [187]:
correlation(df=df,value_cols=['Var1','Var2'])

,Var1,Var2
Var1,1.000000,-0.589734
Var2,-0.589734,1.000000


In [189]:
correlation(df=df,value_cols=['Var1','Var2','Var5'])

,Var1,Var2,Var5
Var1,1.000000,-0.589734,0.254500
Var2,-0.589734,1.000000,-0.063268
Var5,0.254500,-0.063268,1.000000


In [191]:
correlation(df=df,value_cols=['Var1','Var2','Var5'],method='pearson')

,Var1,Var2,Var5
Var1,1.000000,-0.589734,0.254500
Var2,-0.589734,1.000000,-0.063268
Var5,0.254500,-0.063268,1.000000


In [192]:
correlation(df=df,value_cols=['Var1','Var2','Var5'],method='spearman')

,Var1,Var2,Var5
Var1,1.000000,-0.566917,0.246617
Var2,-0.566917,1.000000,-0.030075
Var5,0.246617,-0.030075,1.000000


In [193]:
correlation(df=df,value_cols=['Var1','Var2','Var5'],method='kendall')

,Var1,Var2,Var5
Var1,1.000000,-0.389474,0.168421
Var2,-0.389474,1.000000,-0.021053
Var5,0.168421,-0.021053,1.000000


In [194]:
np.random.seed(42)
df = pd.DataFrame({
    'Var1': np.random.rand(20),
    'Var2': np.random.rand(20),
    'Var3': np.random.rand(20),
    'Var4': np.random.rand(20),
    'Var5': np.random.rand(20),
    'c': ['X', 'X', 'Y', 'Y', 'Z', 'Z', 'A', 'A', 'B', 'B','C', 'C', 'D', 'D', 'E', 'E', 'F', 'F', 'G', 'G'],
    'a': ['pre', 'post', 'pre', 'post', 'pre', 'post','pre', 'post','pre', 'post','pre', 'post','pre', 'post','pre', 'post','pre', 'post','pre', 'post']
})

In [196]:
c = correlation(df,['c','a'],['Var5'])
c

a,post,pre
a,,
post,1.000000,0.495136
pre,0.495136,1.000000
